# 0.0. Imports

In [2]:
# pip install pandas seaborn numpy matplotlib

In [24]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt

import pickle
import os


## 0.1. Helper Functions

In [39]:
def change_date(data, column):
    data[column] = pd.to_datetime(data[column], format='%Y-%m-%d')
    return data[column]

def descrever(df):
    num_attributes = df.select_dtypes(['int64', 'float64'])

    mean = pd.DataFrame(num_attributes.apply(np.mean)).T
    median = pd.DataFrame(num_attributes.apply(np.median)).T

    min_ = pd.DataFrame(num_attributes.apply(np.min)).T
    max_ = pd.DataFrame(num_attributes.apply(np.max)).T
    std = pd.DataFrame(num_attributes.apply(np.std)).T
    range_ =  pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
    skew = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
    kurtosis  = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T
    unique  = pd.DataFrame(num_attributes.apply(lambda x: x.nunique())).T


    metricas = pd.concat([ min_, max_, range_, mean, median, std, skew, kurtosis, unique]).T
    metricas.columns = [ 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis', 'unique']
    return metricas

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)

## 0.2. Load original datasets

In [6]:
df_geral = pd.read_csv('../data/portfolio_geral.csv', encoding='UTF-8')
df_cliente = pd.read_csv('../data/portfolio_clientes.csv', encoding='UTF-8')
df_comunicados = pd.read_csv('../data/portfolio_comunicados.csv', encoding='UTF-8')
df_tpv = pd.read_csv('../data/portfolio_tpv.csv', encoding='UTF-8')


# 1.0. Mudança de tipos

In [7]:
#mudança de tipos

#dates
dates = ['dt_contrato', 'dt_desembolso','dt_vencimento', 'dt_wo', 'dt_ref_portfolio' ]
for date in dates:
    df_geral[date] = change_date(df_geral, date)

df_comunicados['data_acao'] = change_date(df_comunicados, 'data_acao')
df_comunicados['dt_ref_portfolio'] = change_date(df_comunicados, 'dt_ref_portfolio')

df_tpv['dt_transacao'] = change_date(df_tpv, 'dt_transacao')




# 2.0. Tratamentos e Joins

## 2.1. Split portfolio geral

In [8]:
#split geral -> geral e contrato
df_contrato = df_geral[['contrato_id', 'nr_documento', 'safra', 'dt_contrato', 'dt_desembolso',
                        'dt_vencimento','dt_wo', 'prazo', 'vlr_desembolsado',
                        'vlr_tarifa', 'juros_mes','juros_diario']].drop_duplicates('contrato_id')

df_geral = df_geral.drop(['safra', 'dt_contrato', 'dt_desembolso',
                        'dt_vencimento','dt_wo', 'prazo', 'vlr_desembolsado',
                        'vlr_tarifa', 'juros_mes','juros_diario'], axis=1)

## 2.2. Filtrar linhas do df_geral

In [9]:
#filtrar geral -> manter somente primeira linha de settled

#sort df_geral por id e date
df_geral = df_geral.sort_values(['contrato_id', 'dt_ref_portfolio']).reset_index(drop=True)



#pega todas linhas com status quitado
aux = df_geral[df_geral['status_contrato']=='Settled']

#pega data minima de cada contrato quitado
aux0 = aux[['contrato_id', 'dt_ref_portfolio']].groupby('contrato_id').min().reset_index()

#juntar outras features 
aux0 = pd.merge(aux0, df_geral, on=['contrato_id', 'dt_ref_portfolio'], how='inner')

#remover todos status quitados de df_geral
df_geral = df_geral[df_geral['status_contrato']!='Settled']

#colocar de volta apenas o primeiro registro do contrato quitado
df_geral = pd.concat([df_geral, aux0])



In [10]:
#encontrar dados estranhos
descrever(df_geral)

,min,max,range,mean,median,std,skew,kurtosis,unique
perc_retencao,0.0114,1.20,1.1886,0.207289,0.156,0.242711,3.425979,11.127067,2301.0
vlr_pgto_realizado,-3195.5600,210216.25,213411.8100,86.328751,0.000,452.963586,127.412612,35987.554675,111177.0
vlr_pgto_esperado,0.5700,1134.03,1133.4600,112.022508,70.700,124.339717,2.586350,8.586397,12604.0
vlr_saldo_devedor,-44613.3700,750102.75,794716.1200,29216.660267,15216.420,41036.658143,3.663851,21.157096,2878673.0
vlr_saldo_devedor_esperado,0.0000,401338.69,401338.6900,19754.802032,9216.805,30367.406702,3.450363,16.585532,2948472.0
dsp,0.0000,791.00,791.0000,21.725396,1.000,62.933864,4.474875,24.865537,792.0
dspp,0.0000,833.00,833.0000,58.527839,3.000,114.092322,2.513723,6.531199,834.0
flag_transacao,0.0000,1.00,1.0000,0.568374,1.000,0.495303,-0.276088,-1.923776,2.0


In [11]:
#limpar dados estranhos

# vlr_pgto_realizado<0
linhas = len(df_geral[df_geral['vlr_pgto_realizado']<0])
print(f'Linhas removidas para valores de pagamento negativo: {linhas}')
df_geral = df_geral[df_geral['vlr_pgto_realizado']>=0]

# vlr_saldo_devedor <0
linhas = len(df_geral[df_geral['vlr_saldo_devedor']<0])
print(f'Linhas removidas para valores de pagamento negativo: {linhas}')
df_geral = df_geral[df_geral['vlr_saldo_devedor']>=0]


# df_geral[df_geral['vlr_saldo_devedor']<0]

Linhas removidas para valores de pagamento negativo: 23
Linhas removidas para valores de pagamento negativo: 1591


## 2.3. Merge -  portifólio geral e de comunicados

In [12]:
#merge geral e comunicados
df_geral_comunicado = pd.merge(df_geral, df_comunicados, on=['contrato_id', 'dt_ref_portfolio'], how='left')


## 2.4. Merge - portfólio geral e de tpv

In [13]:
#encontrar dados estranhos tpv
descrever(df_tpv)

,min,max,range,mean,median,std,skew,kurtosis,unique
qtd_transacoes,-2.0,1245.00,1247.00,15.906579,7.0,26.153359,6.003932,70.753434,705.0
vlr_tpv,-125000.0,176880.93,301880.93,887.497885,430.0,1664.209969,10.497490,362.253612,492049.0


In [14]:
#limpar dados estranhos
linhas = len(df_tpv[df_tpv['qtd_transacoes']<0])
print(f'Linhas removidas para valores de quantidade de transação negativo: {linhas}')
df_tpv = df_tpv[df_tpv['qtd_transacoes']>=0]

linhas = len(df_tpv[df_tpv['vlr_tpv']<0])
print(f'Linhas removidas para valores de transação de transação negativo: {linhas}')
df_tpv = df_tpv[df_tpv['vlr_tpv']>=0]


Linhas removidas para valores de quantidade de transação negativo: 7
Linhas removidas para valores de transação de transação negativo: 1238


In [15]:
#merge geral e tpv
df_geral = pd.merge(df_geral, df_tpv.rename(columns = {'dt_transacao':'dt_ref_portfolio'}), on=['nr_documento', 'dt_ref_portfolio'], how='left')

## 2.5. Merge - df_contrato e portfólio de clientes


In [16]:
#merge contratos e clientes
df_contrato = pd.merge(df_contrato, df_cliente, on='nr_documento', how='inner')

# 3.0. Feature Engineering

### 3.1. Features em df_geral

In [17]:
#valor debitado desconsiderando pagamento 
df_geral['debito_sem_pagamento'] = df_geral['vlr_saldo_devedor'] + df_geral['vlr_pgto_realizado']


#valor pago tpv = valor * perc_retencao/100
df_geral['valor_pago_tpv'] = df_geral['vlr_tpv'] * df_geral['perc_retencao']/100


### 3.2. Features em df_contrato

In [18]:
#extrair ultima data de cada contrato
last_date = df_geral[['contrato_id', 'dt_ref_portfolio']].groupby('contrato_id').max().reset_index()
last_date = pd.merge(last_date, df_geral[['contrato_id', 'dt_ref_portfolio','status_contrato']], on=['contrato_id', 'dt_ref_portfolio'], how='inner')

# #conseguir o ultimo status de contrato
df_contrato = pd.merge(df_contrato, last_date.drop('dt_ref_portfolio', axis=1), on='contrato_id', how='inner')

#mean dsp / contrato
aux = df_geral[['dsp', 'contrato_id']].groupby('contrato_id').mean().reset_index()
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='left')

#mean dspp / contrato
aux = df_geral[['dspp', 'contrato_id']].groupby('contrato_id').mean().reset_index()
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='left')

#sum vlr pago realizado / contrato
aux = df_geral[['vlr_pgto_realizado', 'contrato_id']].groupby('contrato_id').sum().reset_index()
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='left')

#sum vlr pago tpv realizado / contrato
aux = df_geral[['valor_pago_tpv', 'contrato_id']].groupby('contrato_id').sum().reset_index()
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='left')

#valor total pago / contrato
df_contrato['valor_total_pago'] = df_contrato['vlr_pgto_realizado'] + df_contrato['valor_pago_tpv']

#valor final que deveria ser pago desconsiderando pagamento
aux = df_geral[['debito_sem_pagamento','contrato_id']].groupby('contrato_id').max().reset_index()
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='left')

#dias esperado de contrato
df_contrato['tempo_esperado'] = (df_contrato['dt_vencimento'] - df_contrato['dt_desembolso']).dt.days

# # #dias efetivos de contrato 
aux = df_geral[['contrato_id', 'dt_ref_portfolio']].groupby('contrato_id').count().rename(columns={'dt_ref_portfolio': 'dias_de_contrato'}).reset_index()
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='left')

#acoes totais
aux = df_geral_comunicado[['contrato_id', 'status']].groupby('contrato_id').count().reset_index().rename(columns={'status': 'n_acoes_total'})
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='inner')

# #acoes entregues
aux = df_geral_comunicado [df_geral_comunicado['status']!='NAO ENTREGUE'] [['contrato_id', 'status']].groupby('contrato_id').count().reset_index().rename(columns={'status': 'n_acoes_entregues'})
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='inner')

# 4.0. Criação de tabelas

## 4.1. Criação da df_mensagem

Um dataframe chamado df_mensagem foi criado apenas com as linhas do df_geral que continha uma comunicação com status diferente de “NAO ENTREGUE” e os 5 registros seguintes. O objetivo desse dataframe é observar o efeito da comunicação nos 5 dias seguintes, através do dsp e dspp. Dessa vez o enfoque é descobrir a curva ideal de vezes que se deve acionar o cliente.


In [19]:
# #pegar linha da ação
df_sorted = df_geral_comunicado.sort_values(['contrato_id', 'dt_ref_portfolio']).reset_index(drop=True)

#get index of not null messages
index_mensagem = df_sorted.loc[~df_sorted['status'].isna(), :].index

index_mensagem_expanded = index_mensagem
#create index list expanded

index_aux = index_mensagem + 1
index_mensagem_expanded = index_mensagem_expanded.append(index_aux)
index_aux+=1
index_mensagem_expanded = index_mensagem_expanded.append(index_aux)
index_aux+=1
index_mensagem_expanded = index_mensagem_expanded.append(index_aux)
index_aux+=1
index_mensagem_expanded = index_mensagem_expanded.append(index_aux)
index_aux+=1
index_mensagem_expanded = index_mensagem_expanded.append(index_aux)


#get df mensagem
df_mensagem = df_sorted.iloc[index_mensagem_expanded, :].sort_values(['contrato_id', 'dt_ref_portfolio']).drop_duplicates().reset_index(drop=True)

#get index das mensagens 
index_mensagem = df_mensagem.loc[df_mensagem['status'].isin(['LIDO', 'RESPONDIDO']), :].index

#create eficiencia
for index, i in enumerate(index_mensagem):
#     print(f'{index}/{len(index_mensagem)}')
    
    if df_mensagem.loc[i,'status']: #se status existe

        if df_mensagem.loc[i,'dsp']==0:   #se dsp ==0, passa
            pass

        else:                               #se dsp!=0, verifica os proximos 5 se vai ser igual a 0
            for i2 in range(i+1, i+6):
                if df_mensagem.loc[i2, 'dsp']==0:

                    df_mensagem.loc[i,'eficiencia']=1
    
                    if df_mensagem.loc[i2, 'dspp']==0:
                        df_mensagem.loc[i,'eficiencia']=2
                        
                else: 
                    pass
        

#fill null with 0
df_mensagem['eficiencia'] = df_mensagem['eficiencia'].fillna(0)



## 4.2 Criação da df_lido


Um dataframe chamado df_lido foi criado apenas com as linhas do df_geral que continha uma comunicação com status “LIDO”/”RESPONDIDO” e os 5 registros seguintes. O objetivo desse dataframe é observar o efeito da comunicação nos 5 dias seguintes, através do dsp e dspp.

In [20]:
# #pegar linha da ação
df_sorted = df_geral_comunicado.sort_values(['contrato_id', 'dt_ref_portfolio']).reset_index(drop=True)

#get index of not null messages
index_lido = df_sorted.loc[df_sorted['status'].isin(['LIDO','RESPONDIDO']), :].index

index_lido_expanded = index_lido
#create index list expanded

index_aux = index_lido + 1
index_lido_expanded = index_lido_expanded.append(index_aux)
index_aux+=1
index_lido_expanded = index_lido_expanded.append(index_aux)
index_aux+=1
index_lido_expanded = index_lido_expanded.append(index_aux)
index_aux+=1
index_lido_expanded = index_lido_expanded.append(index_aux)
index_aux+=1
index_lido_expanded = index_lido_expanded.append(index_aux)


#get df lido
df_lido = df_sorted.iloc[index_lido_expanded, :].sort_values(['contrato_id', 'dt_ref_portfolio']).drop_duplicates().reset_index(drop=True)

#get index das mensagens 
index_lido = df_lido.loc[df_lido['status'].isin(['LIDO', 'RESPONDIDO']), :].index

#create eficiencia
for index, i in enumerate(index_lido):
#     print(f'{index}/{len(index_lido)}')
    
    if df_lido.loc[i,'status']: #se status existe

        if df_lido.loc[i,'dsp']==0:   #se dsp ==0, passa
            pass

        else:                               #se dsp!=0, verifica os proximos 5 se vai ser igual a 0
            for i2 in range(i+1, i+6):
                if df_lido.loc[i2, 'dsp']==0:

                    df_lido.loc[i,'eficiencia']=1
    
                    if df_lido.loc[i2, 'dspp']==0:
                        df_lido.loc[i,'eficiencia']=2
                        
                else: 
                    pass
        

#fill null with 0
df_lido['eficiencia'] = df_lido['eficiencia'].fillna(0)



In [21]:
df_lido.head(100)

,contrato_id,dt_ref_portfolio,nr_documento,status_contrato,perc_retencao,vlr_pgto_realizado,vlr_pgto_esperado,vlr_saldo_devedor,vlr_saldo_devedor_esperado,dsp,dspp,flag_transacao,data_acao,tipo_acao,acao,status,eficiencia
0,000180509391a5ac66ff83cae603ffb8,2020-12-29,7996daab1bbe000bb5d1cc1bf317f390,Active,0.060,0.00,27.45,1960.10,4929.02,5,5,0,2020-12-29,EMAIL,campanhaobservacao,LIDO,0.0
1,000180509391a5ac66ff83cae603ffb8,2020-12-29,7996daab1bbe000bb5d1cc1bf317f390,Active,0.060,0.00,27.45,1960.10,4929.02,5,5,0,2020-12-29,HSM,campanhaobservacao,NAO ENTREGUE,0.0
2,000180509391a5ac66ff83cae603ffb8,2020-12-30,7996daab1bbe000bb5d1cc1bf317f390,Active,0.060,0.00,27.45,1965.75,4915.73,6,6,0,NaT,NaN,NaN,NaN,0.0
3,000180509391a5ac66ff83cae603ffb8,2020-12-31,7996daab1bbe000bb5d1cc1bf317f390,Active,0.060,0.00,27.45,1971.39,4902.41,7,7,0,NaT,NaN,NaN,NaN,0.0
4,000180509391a5ac66ff83cae603ffb8,2021-01-01,7996daab1bbe000bb5d1cc1bf317f390,Active,0.060,0.00,27.45,1977.07,4889.06,8,8,0,NaT,NaN,NaN,NaN,0.0
5,000180509391a5ac66ff83cae603ffb8,2021-01-02,7996daab1bbe000bb5d1cc1bf317f390,Active,0.060,0.00,27.45,1982.75,4875.66,9,9,0,NaT,NaN,NaN,NaN,0.0
6,000c35a61297edadc2842f6d5b4028e1,2021-02-13,1191ebfa94d3ca2e8a02f696aafde4a4,Active,0.180,0.00,106.19,25979.28,26170.22,5,5,1,2021-02-15,HSM,campanhaobservacao,LIDO,1.0
7,000c35a61297edadc2842f6d5b4028e1,2021-02-14,1191ebfa94d3ca2e8a02f696aafde4a4,Active,0.180,0.00,106.19,26021.48,26106.53,6,6,1,NaT,NaN,NaN,NaN,0.0
8,000c35a61297edadc2842f6d5b4028e1,2021-02-15,1191ebfa94d3ca2e8a02f696aafde4a4,Active,0.180,0.00,106.19,26063.75,26042.73,7,7,1,NaT,NaN,NaN,NaN,0.0
9,000c35a61297edadc2842f6d5b4028e1,2021-02-16,1191ebfa94d3ca2e8a02f696aafde4a4,Active,0.180,0.00,106.19,26106.09,25978.83,8,8,1,NaT,NaN,NaN,NaN,0.0


## 4.3. Criação do df_inadimplentes

In [22]:
df_inadimplentes = df_contrato[df_contrato['status_contrato']!='Settled']

# 5.0. Exportar para csv

In [23]:
df_contrato.to_csv('../data/new_dfs/df_contrato.csv')
df_geral.to_csv('../data/new_dfs/df_geral.csv')
df_geral_comunicado.to_csv('../data/new_dfs/df_geral_comunicado.csv')
df_lido.to_csv('../data/new_dfs/df_lido.csv')
df_mensagem.to_csv('../data/new_dfs/df_mensagem.csv')
df_inadimplentes.to_csv('../data/new_dfs/df_inadimplentes.csv')

# 6.0. DataFrame previews

In [40]:
df_geral.head()

,contrato_id,dt_ref_portfolio,nr_documento,status_contrato,perc_retencao,vlr_pgto_realizado,vlr_pgto_esperado,vlr_saldo_devedor,vlr_saldo_devedor_esperado,dsp,dspp,flag_transacao,qtd_transacoes,vlr_tpv,debito_sem_pagamento,valor_pago_tpv
0,000180509391a5ac66ff83cae603ffb8,2020-06-15,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,0.00,27.45,6932.34,6924.81,0,0,0,NaN,NaN,6932.34,NaN
1,000180509391a5ac66ff83cae603ffb8,2020-06-16,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,0.00,27.45,6952.26,6917.27,1,1,1,NaN,NaN,6952.26,NaN
2,000180509391a5ac66ff83cae603ffb8,2020-06-17,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,25.43,27.45,6946.81,6909.70,0,2,1,NaN,NaN,6972.24,NaN
3,000180509391a5ac66ff83cae603ffb8,2020-06-18,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,41.40,27.45,6925.38,6902.11,0,0,1,NaN,NaN,6966.78,NaN
4,000180509391a5ac66ff83cae603ffb8,2020-06-19,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,61.49,27.45,6883.79,6894.50,0,0,1,NaN,NaN,6945.28,NaN


In [41]:
df_contrato.head()

,contrato_id,nr_documento,safra,dt_contrato,dt_desembolso,dt_vencimento,dt_wo,prazo,vlr_desembolsado,vlr_tarifa,juros_mes,juros_diario,tipo_empresa,cidade,estado,subsegmento,segmento,status_contrato,dsp,dspp,vlr_pgto_realizado,valor_pago_tpv,valor_total_pago,debito_sem_pagamento,tempo_esperado,dias_de_contrato,n_acoes_total,n_acoes_entregues
0,356c02706c8e74b15004bb5964ade6bb,b0871d4e0d72afd1d44381e5b2453cb2,2020-06,2020-06-24,2020-06-29,2020-12-24,2021-12-24,10.26,22686.57,0.0,0.10788,0.00345,PF,João Pessoa,PB,Educação,Serviços recorrentes,Settled,0.722222,0.722222,24673.98,0.0,24673.98,22751.75,178,54,0,0
1,a5940634cbbe2a6d01d0676f6dfd6368,b0871d4e0d72afd1d44381e5b2453cb2,2020-08,2020-08-24,2020-08-26,2022-02-24,2023-02-24,30.78,59018.94,0.0,0.09588,0.00308,PF,João Pessoa,PB,Educação,Serviços recorrentes,Active,34.474210,74.061564,58600.16,0.0,58600.16,71562.83,547,601,58,30
2,60a0f8d19ec695ea4f79710212cfd6dd,b0871d4e0d72afd1d44381e5b2453cb2,2021-03,2021-03-02,2021-03-04,2022-07-02,2023-07-02,27.36,25752.60,0.0,0.09588,0.00308,PF,João Pessoa,PB,Educação,Serviços recorrentes,Active,56.795620,64.364964,12761.39,0.0,12761.39,39264.01,485,411,70,41
3,1eb0e13f83f336146227ccc9efb08c47,938513461b4a4fcd2bf99ffd8f12b9be,2020-07,2020-07-26,2020-07-28,2021-10-26,2022-10-26,25.65,16074.00,0.0,0.04788,0.00157,MEI,Campo Grande,MS,Alimentação Rápida,Alimentação,Settled,0.598101,0.727848,19954.35,0.0,19954.35,16113.74,455,316,0,0
4,8b03675f597974ca96f06648e96af6d4,4303bd71ee3fe80aba4efc8dbd6c7d70,2020-08,2020-08-13,2020-08-17,2022-02-13,2023-02-13,30.78,17100.00,0.0,0.05388,0.00176,PJ,Mogi Mirim,SP,Outros,Serviços,Settled,0.387029,0.414226,22360.45,0.0,22360.45,17125.05,545,478,0,0


In [42]:
df_geral_comunicado.head()

,contrato_id,dt_ref_portfolio,nr_documento,status_contrato,perc_retencao,vlr_pgto_realizado,vlr_pgto_esperado,vlr_saldo_devedor,vlr_saldo_devedor_esperado,dsp,dspp,flag_transacao,data_acao,tipo_acao,acao,status
0,000180509391a5ac66ff83cae603ffb8,2020-06-15,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,0.00,27.45,6932.34,6924.81,0,0,0,NaT,NaN,NaN,NaN
1,000180509391a5ac66ff83cae603ffb8,2020-06-16,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,0.00,27.45,6952.26,6917.27,1,1,1,NaT,NaN,NaN,NaN
2,000180509391a5ac66ff83cae603ffb8,2020-06-17,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,25.43,27.45,6946.81,6909.70,0,2,1,NaT,NaN,NaN,NaN
3,000180509391a5ac66ff83cae603ffb8,2020-06-18,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,41.40,27.45,6925.38,6902.11,0,0,1,NaT,NaN,NaN,NaN
4,000180509391a5ac66ff83cae603ffb8,2020-06-19,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,61.49,27.45,6883.79,6894.50,0,0,1,NaT,NaN,NaN,NaN


In [43]:
df_lido.head()

,contrato_id,dt_ref_portfolio,nr_documento,status_contrato,perc_retencao,vlr_pgto_realizado,vlr_pgto_esperado,vlr_saldo_devedor,vlr_saldo_devedor_esperado,dsp,dspp,flag_transacao,data_acao,tipo_acao,acao,status,eficiencia
0,000180509391a5ac66ff83cae603ffb8,2020-12-29,7996daab1bbe000bb5d1cc1bf317f390,Active,0.06,0.0,27.45,1960.10,4929.02,5,5,0,2020-12-29,EMAIL,campanhaobservacao,LIDO,0.0
1,000180509391a5ac66ff83cae603ffb8,2020-12-29,7996daab1bbe000bb5d1cc1bf317f390,Active,0.06,0.0,27.45,1960.10,4929.02,5,5,0,2020-12-29,HSM,campanhaobservacao,NAO ENTREGUE,0.0
2,000180509391a5ac66ff83cae603ffb8,2020-12-30,7996daab1bbe000bb5d1cc1bf317f390,Active,0.06,0.0,27.45,1965.75,4915.73,6,6,0,NaT,NaN,NaN,NaN,0.0
3,000180509391a5ac66ff83cae603ffb8,2020-12-31,7996daab1bbe000bb5d1cc1bf317f390,Active,0.06,0.0,27.45,1971.39,4902.41,7,7,0,NaT,NaN,NaN,NaN,0.0
4,000180509391a5ac66ff83cae603ffb8,2021-01-01,7996daab1bbe000bb5d1cc1bf317f390,Active,0.06,0.0,27.45,1977.07,4889.06,8,8,0,NaT,NaN,NaN,NaN,0.0


In [44]:
df_mensagem.head()

,contrato_id,dt_ref_portfolio,nr_documento,status_contrato,perc_retencao,vlr_pgto_realizado,vlr_pgto_esperado,vlr_saldo_devedor,vlr_saldo_devedor_esperado,dsp,dspp,flag_transacao,data_acao,tipo_acao,acao,status,eficiencia
0,000180509391a5ac66ff83cae603ffb8,2020-12-29,7996daab1bbe000bb5d1cc1bf317f390,Active,0.06,0.0,27.45,1960.10,4929.02,5,5,0,2020-12-29,EMAIL,campanhaobservacao,LIDO,0.0
1,000180509391a5ac66ff83cae603ffb8,2020-12-29,7996daab1bbe000bb5d1cc1bf317f390,Active,0.06,0.0,27.45,1960.10,4929.02,5,5,0,2020-12-29,HSM,campanhaobservacao,NAO ENTREGUE,0.0
2,000180509391a5ac66ff83cae603ffb8,2020-12-30,7996daab1bbe000bb5d1cc1bf317f390,Active,0.06,0.0,27.45,1965.75,4915.73,6,6,0,NaT,NaN,NaN,NaN,0.0
3,000180509391a5ac66ff83cae603ffb8,2020-12-31,7996daab1bbe000bb5d1cc1bf317f390,Active,0.06,0.0,27.45,1971.39,4902.41,7,7,0,NaT,NaN,NaN,NaN,0.0
4,000180509391a5ac66ff83cae603ffb8,2021-01-01,7996daab1bbe000bb5d1cc1bf317f390,Active,0.06,0.0,27.45,1977.07,4889.06,8,8,0,NaT,NaN,NaN,NaN,0.0


In [45]:
df_inadimplentes.head()

,contrato_id,nr_documento,safra,dt_contrato,dt_desembolso,dt_vencimento,dt_wo,prazo,vlr_desembolsado,vlr_tarifa,juros_mes,juros_diario,tipo_empresa,cidade,estado,subsegmento,segmento,status_contrato,dsp,dspp,vlr_pgto_realizado,valor_pago_tpv,valor_total_pago,debito_sem_pagamento,tempo_esperado,dias_de_contrato,n_acoes_total,n_acoes_entregues
1,a5940634cbbe2a6d01d0676f6dfd6368,b0871d4e0d72afd1d44381e5b2453cb2,2020-08,2020-08-24,2020-08-26,2022-02-24,2023-02-24,30.78,59018.94,0.0,0.09588,0.00308,PF,João Pessoa,PB,Educação,Serviços recorrentes,Active,34.474210,74.061564,58600.16,0.0,58600.16,71562.83,547,601,58,30
2,60a0f8d19ec695ea4f79710212cfd6dd,b0871d4e0d72afd1d44381e5b2453cb2,2021-03,2021-03-02,2021-03-04,2022-07-02,2023-07-02,27.36,25752.60,0.0,0.09588,0.00308,PF,João Pessoa,PB,Educação,Serviços recorrentes,Active,56.795620,64.364964,12761.39,0.0,12761.39,39264.01,485,411,70,41
5,4207c25cdd13aba5a05ae348e889d798,9c1dc4403f8a4833bd58eea3ee85c917,2021-03,2021-03-01,2021-03-03,2022-06-01,2023-06-01,25.65,14588.01,0.0,0.04788,0.00157,PJ,Amparo,SP,Automotivo,Bens duráveis,Active,5.077670,28.985437,11215.07,0.0,11215.07,14729.21,455,412,68,39
6,7dfdca96b6ff0dd844e5e130aee88b30,3b6951df3075979c99a4dfd9f93efd38,2020-11,2020-10-31,2020-11-04,2022-01-31,2023-01-31,25.65,68400.00,0.0,0.04788,0.00157,PJ,Luziânia,GO,Alimentação Rápida,Alimentação,Active,32.214689,45.210923,44533.54,0.0,44533.54,68569.77,453,531,30,17
7,a3aecc6b029cc77dd867e0e24e2d616f,9740eda3d6190add5703cd2dd280023c,2020-12,2020-12-18,2020-12-22,2022-06-18,2023-06-18,30.78,36380.25,0.0,0.04188,0.00137,PF,Santa Rita,PB,Vestuário,Varejo,Active,62.202899,92.826087,15427.53,0.0,15427.53,38838.55,543,483,20,14
